In [1]:
import os
from bs4 import BeautifulSoup
from splinter import Browser
import requests
from selenium import webdriver
import pandas as pd
import pymongo
import json
from bson import json_util
from selenium import webdriver

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection
db = client.giants_db
collection = db.players

In [4]:
#https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [5]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [6]:
url = "http://mlb.mlb.com/stats/sortable.jsp"
browser.visit(url)



In [11]:
# for x in range(40):
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

# Retrieve all elements that contain book information
# table = soup.find('table', class_="stats_table data_grid")    # THIS IS YOUR OLD CODE
table = soup.find('table', class_="stats_table data_grid").find('tbody')


players = []
# rowcount = 0    # DEBUGGING
for tr in table:
#    print(f"****************** ROW {rowcount} *************")    # DEBUGGING
#    rowcount += 1    # DEBUGGING
   tds = tr.find_all("td")

#    print(tds[0].get_text())   # DEBUGGING
   player = {}
#    tdcount=0    # DEBUGGING
   for td in tds:
#        print(f"-----------------TD {tdcount} -----------------------")    # DEBUGGING
#        tdcount +=1   # DEBUGGING
#        print(td.get_text(), td['class'][0])   # DEBUGGING
       cls = td['class'][0]
       if cls == 'dg-name_display_last_init':
            player.update(name = td.text)
       if cls == 'dg-pos':
           player.update(position = td.text)
       if cls == 'dg-g':
           player.update(Games_played = td.text)
       if cls == 'dg-ab':
           player.update(At_Bat = td.text)
       if cls == 'dg-h':
           player.update(Hits = td.text)
       if cls == 'dg-avg':
           player.update(Average = td.text)
       if cls == 'dg-obp':
           player.update(Player_reaches_base = td.text)
   if len(player) > 0:
       players.append(player)
   player = {}
print(players)# DEBUGGING



data = pd.DataFrame.from_dict(players, orient='columns')




[{'name': '\xa0Law, D', 'position': 'P', 'Games_played': '7', 'At_Bat': '1', 'Hits': '1', 'Average': '1.000', 'Player_reaches_base': '1.000'}, {'name': '\xa0Jones, R', 'position': '3B', 'Games_played': '5', 'At_Bat': '8', 'Hits': '3', 'Average': '.375', 'Player_reaches_base': '.375'}, {'name': '\xa0Garcia, A', 'position': 'C', 'Games_played': '19', 'At_Bat': '63', 'Hits': '18', 'Average': '.286', 'Player_reaches_base': '.308'}, {'name': '\xa0Posey, B', 'position': 'C', 'Games_played': '105', 'At_Bat': '398', 'Hits': '113', 'Average': '.284', 'Player_reaches_base': '.359'}, {'name': '\xa0Avelino, A', 'position': 'SS', 'Games_played': '6', 'At_Bat': '11', 'Hits': '3', 'Average': '.273', 'Player_reaches_base': '.273'}, {'name': '\xa0Gomez, M', 'position': '2B', 'Games_played': '9', 'At_Bat': '15', 'Hits': '4', 'Average': '.267', 'Player_reaches_base': '.267'}, {'name': '\xa0Duggar, S', 'position': 'CF', 'Games_played': '41', 'At_Bat': '141', 'Hits': '36', 'Average': '.255', 'Player_reache

In [12]:
print(data)

   At_Bat Average Games_played Hits Player_reaches_base            name  \
0       1   1.000            7    1               1.000          Law, D   
1       8    .375            5    3                .375        Jones, R   
2      63    .286           19   18                .308       Garcia, A   
3     398    .284          105  113                .359        Posey, B   
4      11    .273            6    3                .273      Avelino, A   
5      15    .267            9    4                .267        Gomez, M   
6     141    .255           41   36                .303       Duggar, S   
7     482    .255          130  123                .357    McCutchen, A   
8     531    .254          151  135                .325     Crawford, B   
9     358    .254          102   91                .307        Panik, J   
10    399    .253          112  101                .342         Belt, B   
11    294    .252          110   74                .274       Hanson, A   
12    199    .251        

In [19]:
records = json.loads(data.to_json(orient='records'))
type(records)


#mongoImp = collection.insert_many(jArray)


list

In [20]:
#insert into database
collection.insert_many(records)

In [18]:
#records.delete({})

AttributeError: 'list' object has no attribute 'delete'